In [ ]:
%%sh
pip install -U pip
pip install -qU yfinance openai

In [1]:
import json
import pprint

import yfinance
from IPython.display import Markdown, clear_output, display
from openai import OpenAI
from print_streaming_response import print_streaming_response
from api_key import api_key

endpoint = "https://conductor.arcee.ai/v1"

client = OpenAI(
    base_url=endpoint,
    api_key=api_key
)

In [2]:
def get_stock_price(company_name, stock_symbol):
    stock = yfinance.Ticker(stock_symbol)
    price = stock.history(period="1d")["Close"].values[0]
    return (
        f"The last closing price of {company_name} ({stock_symbol}) was ${price:.2f}."
    )


def get_ceo_name(company_name, stock_symbol):
    stock = yfinance.Ticker(stock_symbol)
    info = stock.info
    ceo = info["companyOfficers"][0]["name"]
    return f"The CEO of {company_name} is {ceo}. The full job title is {info['companyOfficers'][0]['title']}."


def get_company_summary(company_name, stock_symbol):
    stock = yfinance.Ticker(stock_symbol)
    summary = stock.info["longBusinessSummary"]
    return summary

In [10]:
get_stock_price("Amazon", "AMZN")

'The last closing price of Amazon (AMZN) was $213.48.'

In [11]:
get_ceo_name("Ford", "F")

'The CEO of Ford is Mr. William Clay Ford Jr.. The full job title is Executive Chairman.'

In [3]:
get_company_summary("Verizon", "VZ")

'Verizon Communications Inc., through its subsidiaries, engages in the provision of communications, technology, information, and entertainment products and services to consumers, businesses, and governmental entities worldwide. It operates in two segments, Verizon Consumer Group (Consumer) and Verizon Business Group (Business). The Consumer segment provides wireless services across the wireless networks in the United States under the Verizon and TracFone brands and through wholesale and other arrangements; and fixed wireless access (FWA) broadband through its wireless networks, as well as related equipment and devices, such as smartphones, tablets, smart watches, and other wireless-enabled connected devices. The segment also offers wireline services in the Mid-Atlantic and Northeastern United States, as well as Washington D.C. through its fiber-optic network, Verizon Fios product portfolio, and a copper-based network. The Business segment provides wireless and wireline communications s

In [4]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_stock_price",
            "description": """Retrieves the most recent closing stock price that can answer questions about:
- Current stock price
- Recent stock performance
- Company market value
- Share price information

Use this function for questions like:
- "What's the stock price of [company]?"
- "How much does [company] stock cost?"
- "What's [company]'s current share price?"
- "How much are shares of [company] trading for?"
- "What's the latest price for [stock symbol]?"
""",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The full company name (e.g., 'Apple Inc.', 'Tesla, Inc.', 'McDonald's Corporation')",
                    },
                    "stock_symbol": {
                        "type": "string",
                        "description": "The stock market ticker symbol (e.g., 'AAPL', 'TSLA', 'MCD')",
                    },
                },
                "required": ["company_name", "stock_symbol"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_ceo_name",
            "description": """Retrieves current leadership information that can answer questions about:
- CEO name and title
- Top executive information
- Company leadership
- Management details

Use this function for questions like:
- "Who is the CEO of [company]?"
- "Who runs [company]?"
- "What's the name of [company]'s chief executive?"
- "Who is in charge of [company]?"
- "Who is [company]'s leader?"
""",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The full company name (e.g., 'Microsoft Corporation', 'Meta Platforms, Inc.', 'Ford Motor Company')",
                    },
                    "stock_symbol": {
                        "type": "string",
                        "description": "The stock market ticker symbol (e.g., 'MSFT', 'META', 'F')",
                    },
                },
                "required": ["company_name", "stock_symbol"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_company_summary",
            "description": """Retrieves a detailed business summary that can answer questions about:
- Products and services offered
- Target markets and industries served
- Manufacturing and production capabilities
- Business divisions and segments
- Geographic operations
- Company history and background
- Competitive position
- Core business activities

Use this function for questions like:
- "Does [company] make [product]?"
- "What industries does [company] serve?"
- "What are [company]'s main products?"
- "Does [company] operate in [industry/market]?"
- "What does [company] do?"
- "Tell me about [company]'s business"
- "What kind of company is [company]?"
""",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The full company name (e.g., '3M Company', 'Johnson & Johnson', 'The Coca-Cola Company')",
                    },
                    "stock_symbol": {
                        "type": "string",
                        "description": "The stock market ticker symbol (e.g., 'MMM', 'JNJ', 'KO')",
                    },
                },
                "required": ["company_name", "stock_symbol"],
            },
        },
    },
]

In [5]:
def call_tools(user_prompt, max_tokens=128):
    response = client.chat.completions.create(
        model="auto-tool",
        messages=[{"role": "user", "content": user_prompt}],
        tools=tools,
        tool_choice="auto",
        max_tokens=max_tokens,
    )

    # Extract and print the model name from the response
    model_name = response.model
    print(f"Model used: {model_name}")

    tool_calls = response.choices[0].message.tool_calls
    results = []

    # Check if there are any tool calls
    if tool_calls:
        for tool_call in tool_calls:
            # Extract function name and arguments
            function_name = tool_call.function.name
            arguments_json = tool_call.function.arguments
            arguments_dict = json.loads(arguments_json)

            if function_name in globals():
                # Get the function object based on its name
                function_to_call = globals()[function_name]

                print(f"Calling {function_name} with arguments: {arguments_dict}")

                # Call the function with unpacked keyword arguments
                result = function_to_call(**arguments_dict)
                results.append(result)
            else:
                print(f"Function {function_name} not found in the global namespace.")
                results.append(None)

        return results
    else:
        # No tool call: print the generated response
        print("No tool call")
        return None

In [6]:
def call_tools_and_invoke_model(model, user_prompt, max_tokens=1024):
    tool_results = call_tools(user_prompt)

    # Convert list of results to string if we got multiple results
    tool_results_str = (
        "\n".join(tool_results) if isinstance(tool_results, list) else tool_results
    )

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": """You are an expert financial and business analyst providing precise, well-researched answers.

Your capabilities:
- Analyze company information, financials, and market data
- Compare companies and their products/services
- Provide insights about business strategies and market positions
- Explain industry trends and competitive dynamics

Guidelines:
1. Always start with analyzing the tool result if available
2. Structure your response clearly using markdown
3. Be detailed and thorough
4. Include relevant context from your knowledge
5. Support claims with data and sources when possible
6. Stay neutral and objective

Required sections:
* Tool information - Did you use the tool or not?
* Direct Answer - Address the main question first.
* Analysis - Provide context and deeper insights
* Resources - if applicable, include for all companies:
   - Top-level company website
   - Yahoo Finance page (if the company is listed)
   - URL for company SEC filings (if the company is publicly traded)
   - URL for Google search based on the user question

If the tool result is unavailable or insufficient, acknowledge this first and proceed with your best analysis based on available knowledge.""",
            },
            {
                "role": "user",
                "content": f"""
                    question: {user_prompt}
                    tool results: {tool_results_str}
                """,
            },
        ],
        max_tokens=max_tokens,
        stream=True,
    )
    return response

In [7]:
user_prompt = "Who's the CEO of General Motors?"
#user_prompt = "What's the last price of Mc Donalds?"
#user_prompt = "Does 3M make filtration products for the automotive industry?"
#user_prompt = "On which products do Procter and Gamble and Johnson & Johnson compete the most?"
#user_prompt = "What's the population of the capital of New Zealand?"

In [8]:
response = call_tools(user_prompt)
pprint.pprint(response)

Model used: arcee-ai/caller
Calling get_ceo_name with arguments: {'company_name': 'General Motors', 'stock_symbol': 'GM'}
['The CEO of General Motors is Ms. Mary T. Barra. The full job title is '
 'Chairman & CEO.']


In [9]:
response = call_tools_and_invoke_model(model="auto", user_prompt=user_prompt)
print_streaming_response(response)

### Tool Information
Yes, I used the tool to gather the information about the CEO of General Motors.

### Direct Answer
The CEO of General Motors is Ms. Mary T. Barra. Her full job title is Chairman & CEO.

### Analysis
Mary T. Barra has been the CEO of General Motors since 2014, making her the first female CEO of a major U.S. automaker. She has a long-standing career with GM, having joined the company in 1980. Prior to her role as CEO, she served in various positions within the company, including Executive Vice President of Global Product Development, Purchasing, and Supply Chain.

Under her leadership, General Motors has focused on innovation, particularly in the areas of electric vehicles and autonomous driving technology. GM's strategy includes significant investments in electric vehicle production and the development of the Ultium battery platform, which is a part of the company's broader commitment to an all-electric future. Barra’s vision is to position GM as a leader in the transition to a sustainable transportation ecosystem.

In addition to her role at GM, Mary Barra is a prominent figure in the business community, often seen as a trailblazer for women in leadership roles in the automotive industry. She has been recognized for her contributions to the industry and her influence as a corporate leader.

### Resources
- **Top-level company website**: [General Motors](https://www.gm.com)
- **Yahoo Finance page**: [General Motors on Yahoo Finance](https://finance.yahoo.com/quote/GM)
- **URL for company SEC filings**: [General Motors SEC Filings](https://www.sec.gov/cgi-bin/browse-edgar?company=General+Motors+Company&CIK=000010610C)
- **URL for Google search**: [Google search for Mary T. Barra CEO General Motors](https://www.google.com/search?q=Mary+T.+Barra+CEO+General+Motors)



Number of tokens: 412
Model ID: togethercomputer/Arcee-Blitz
